In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

In [2]:
df = pd.read_csv("static/data/water.csv")
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pwsId,year,analyteCode,pwsName,countyServed,cityServed,waterSource,mean,max,populationServed,fips
0,1010001,1998.0,Arsenic,Aitkin,Aitkin,Aitkin,Groundwater,3.35 ug/L,3.6 ug/L,1984.0,27001.0
1,1010002,1998.0,Arsenic,Thompson Mobile Home Park,Aitkin,Aitkin,Groundwater,0.5 ug/L (ND),0.5 ug/L (ND),45.0,27001.0
2,1010019,1998.0,Arsenic,Why-Not Park,Aitkin,Aitkin,Groundwater,NaN,NaN,25.0,27001.0
3,1010011,1998.0,Arsenic,Hill City,Aitkin,Hill City,Groundwater,4.4 ug/L,4.4 ug/L,633.0,27001.0
4,1010016,1998.0,Arsenic,McGregor,Aitkin,McGregor,Groundwater,NaN,NaN,394.0,27001.0


In [3]:
df_1 = df[["year", "countyServed", "populationServed"]]
df_1.head()

,year,countyServed,populationServed
0,1998.0,Aitkin,1984.0
1,1998.0,Aitkin,45.0
2,1998.0,Aitkin,25.0
3,1998.0,Aitkin,633.0
4,1998.0,Aitkin,394.0


In [4]:
df_2 = df_1.loc[df_1["year"]==2017]
df_2.head()

,year,countyServed,populationServed
17248,2017.0,Aitkin,2165.0
17249,2017.0,Aitkin,45.0
17250,2017.0,Aitkin,25.0
17251,2017.0,Aitkin,633.0
17252,2017.0,Aitkin,394.0


In [5]:
df_3 = df_2.groupby(['countyServed']).mean()
df_3.head()

,year,populationServed
countyServed,,
Aitkin,2017.0,628.157895
Anoka,2017.0,9339.285714
Becker,2017.0,1605.960784
Beltrami,2017.0,1293.255814
Benton,2017.0,2868.790698


In [6]:
df_4 = df_3.reset_index()
df_4.head()

,countyServed,year,populationServed
0,Aitkin,2017.0,628.157895
1,Anoka,2017.0,9339.285714
2,Becker,2017.0,1605.960784
3,Beltrami,2017.0,1293.255814
4,Benton,2017.0,2868.790698


In [7]:
df_4.round(0)

,countyServed,year,populationServed
0,Aitkin,2017.0,628.0
1,Anoka,2017.0,9339.0
2,Becker,2017.0,1606.0
3,Beltrami,2017.0,1293.0
4,Benton,2017.0,2869.0
...,...,...,...
82,Watonwan,2017.0,1061.0
83,Wilkin,2017.0,936.0
84,Winona,2017.0,3449.0
85,Wright,2017.0,5228.0


In [8]:
df_5 = df_4.rename(columns={"countyServed" : "County", "year" : "Year", "populationServed": "Population"})
df_5

,County,Year,Population
0,Aitkin,2017.0,628.157895
1,Anoka,2017.0,9339.285714
2,Becker,2017.0,1605.960784
3,Beltrami,2017.0,1293.255814
4,Benton,2017.0,2868.790698
...,...,...,...
82,Watonwan,2017.0,1060.629630
83,Wilkin,2017.0,936.290323
84,Winona,2017.0,3449.000000
85,Wright,2017.0,5228.439024


In [9]:
df_5.dtypes

County         object
Year          float64
Population    float64
dtype: object

In [10]:
df_5["Year"] = df_5["Year"].astype(int)
df_5

,County,Year,Population
0,Aitkin,2017,628.157895
1,Anoka,2017,9339.285714
2,Becker,2017,1605.960784
3,Beltrami,2017,1293.255814
4,Benton,2017,2868.790698
...,...,...,...
82,Watonwan,2017,1060.629630
83,Wilkin,2017,936.290323
84,Winona,2017,3449.000000
85,Wright,2017,5228.439024


In [11]:
df_5["Population"] = df_5["Population"].astype(int)
df_5

,County,Year,Population
0,Aitkin,2017,628
1,Anoka,2017,9339
2,Becker,2017,1605
3,Beltrami,2017,1293
4,Benton,2017,2868
...,...,...,...
82,Watonwan,2017,1060
83,Wilkin,2017,936
84,Winona,2017,3449
85,Wright,2017,5228


In [12]:
df_5.to_csv("static/data/countypop_2017.csv")

Reading in geoJson

In [13]:
mn_df = "static/data/mn_water_data-coord.geojson"
mn_countydf = gpd.read_file(mn_df)
mn_countydf

,name,analyteCode,year,mean_value,max_value,fips,geometry
0,Aitkin,Arsenic,2017,3.4,3.5,27001,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1,Aitkin,HAA5,2017,2.3,2.3,27001,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
2,Aitkin,Nitrate,2017,0.0,0.0,27001,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
3,Aitkin,PCE,2017,0.4,0.4,27001,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
4,Aitkin,Radium,2017,0.7,0.7,27001,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
...,...,...,...,...,...,...,...
642,Yellow Medicine,PCE,2017,0.2,0.2,27173,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
643,Yellow Medicine,Radium,2017,0.5,0.5,27173,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
644,Yellow Medicine,TCE,2017,0.1,0.1,27173,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
645,Yellow Medicine,TTHM,2017,7.4,11.9,27173,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."


In [64]:
mn_countydf1 = mn_countydf[["name", "year", "geometry"]]
mn_countydf1

,name,year,geometry
0,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
2,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
3,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
4,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
...,...,...,...
642,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
643,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
644,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
645,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."


In [72]:
mn_countydf2 = mn_countydf1.rename(columns={"name":"County", "year":"Year"})
mn_countydf2

,County,Year,geometry
0,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
2,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
3,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
4,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
...,...,...,...
642,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
643,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
644,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."
645,Yellow Medicine,2017,"POLYGON ((-96.45123 44.71827, -96.45126 44.722..."


In [78]:
mn_countydf2 = mn_countydf2.drop_duplicates()
mn_countydf2

,County,Year,geometry
0,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
7,Anoka,2017,"POLYGON ((-93.51099 45.29935, -93.51097 45.300..."
15,Becker,2017,"POLYGON ((-96.17431 46.89165, -96.17430 46.891..."
22,Beltrami,2017,"POLYGON ((-95.59382 48.19455, -95.59382 48.194..."
29,Benton,2017,"POLYGON ((-94.24770 45.73522, -94.24784 45.735..."
...,...,...,...
608,Watonwan,2017,"POLYGON ((-94.85922 44.02128, -94.85924 44.022..."
616,Wilkin,2017,"POLYGON ((-96.40435 46.63075, -96.40428 46.630..."
623,Winona,2017,"POLYGON ((-92.07851 44.02033, -92.07860 44.023..."
631,Wright,2017,"POLYGON ((-94.25668 45.23927, -94.25699 45.239..."


# Merge

In [84]:
geopop = mn_countydf2.merge(df_5, on="County")
geopop

,County,Year_x,geometry,Year_y,Population
0,Aitkin,2017,"POLYGON ((-93.77799 46.59493, -93.77799 46.595...",2017,628
1,Anoka,2017,"POLYGON ((-93.51099 45.29935, -93.51097 45.300...",2017,9339
2,Becker,2017,"POLYGON ((-96.17431 46.89165, -96.17430 46.891...",2017,1605
3,Beltrami,2017,"POLYGON ((-95.59382 48.19455, -95.59382 48.194...",2017,1293
4,Benton,2017,"POLYGON ((-94.24770 45.73522, -94.24784 45.735...",2017,2868
...,...,...,...,...,...
82,Watonwan,2017,"POLYGON ((-94.85922 44.02128, -94.85924 44.022...",2017,1060
83,Wilkin,2017,"POLYGON ((-96.40435 46.63075, -96.40428 46.630...",2017,936
84,Winona,2017,"POLYGON ((-92.07851 44.02033, -92.07860 44.023...",2017,3449
85,Wright,2017,"POLYGON ((-94.25668 45.23927, -94.25699 45.239...",2017,5228


In [85]:
geopop1 = geopop[["County", "Year_x", "Population", "geometry"]]
geopop1

,County,Year_x,Population,geometry
0,Aitkin,2017,628,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1,Anoka,2017,9339,"POLYGON ((-93.51099 45.29935, -93.51097 45.300..."
2,Becker,2017,1605,"POLYGON ((-96.17431 46.89165, -96.17430 46.891..."
3,Beltrami,2017,1293,"POLYGON ((-95.59382 48.19455, -95.59382 48.194..."
4,Benton,2017,2868,"POLYGON ((-94.24770 45.73522, -94.24784 45.735..."
...,...,...,...,...
82,Watonwan,2017,1060,"POLYGON ((-94.85922 44.02128, -94.85924 44.022..."
83,Wilkin,2017,936,"POLYGON ((-96.40435 46.63075, -96.40428 46.630..."
84,Winona,2017,3449,"POLYGON ((-92.07851 44.02033, -92.07860 44.023..."
85,Wright,2017,5228,"POLYGON ((-94.25668 45.23927, -94.25699 45.239..."


In [86]:
geopop2 = geopop1.rename(columns={"Year_x":"Year"})
geopop2

,County,Year,Population,geometry
0,Aitkin,2017,628,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1,Anoka,2017,9339,"POLYGON ((-93.51099 45.29935, -93.51097 45.300..."
2,Becker,2017,1605,"POLYGON ((-96.17431 46.89165, -96.17430 46.891..."
3,Beltrami,2017,1293,"POLYGON ((-95.59382 48.19455, -95.59382 48.194..."
4,Benton,2017,2868,"POLYGON ((-94.24770 45.73522, -94.24784 45.735..."
...,...,...,...,...
82,Watonwan,2017,1060,"POLYGON ((-94.85922 44.02128, -94.85924 44.022..."
83,Wilkin,2017,936,"POLYGON ((-96.40435 46.63075, -96.40428 46.630..."
84,Winona,2017,3449,"POLYGON ((-92.07851 44.02033, -92.07860 44.023..."
85,Wright,2017,5228,"POLYGON ((-94.25668 45.23927, -94.25699 45.239..."


In [87]:
geopop2.to_file("static/data/geopop.geojson", driver='GeoJSON')